# GSM8K

In [ ]:
import json
import os
import openai
import re
from datasets import load_dataset
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# Load the GSM8K dataset
dataset = load_dataset("gsm8k", "main")

# Directory to save results
output_dir = 'results'
os.makedirs(output_dir, exist_ok=True)

def call_gpt(question):
    """Call OpenAI's GPT model with a question and return the response."""
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",  # Use your desired model
                messages=[
                    {'role': "system", "content": "You are a helpful assistant."},
                    {'role': "user", "content": question},
                ],
                temperature=0,
                max_tokens=100,  # Adjust based on expected answer length
                n=1,
            )
            return response['choices'][0]['message']['content'].strip()
        except (openai.error.APIError, openai.error.RateLimitError, 
                openai.error.ServiceUnavailableError, openai.error.Timeout) as e:
            print(f"Error: {e}. Retrying in 2 seconds...")
            time.sleep(2)

# Main execution loop
results = []
for instance in tqdm(dataset):
    question = instance['question']
    answer = call_gpt(question)
    
    # Store the question and answer
    results.append({
        "question": question,
        "answer": answer
    })

# Save results to a JSON file
with open(os.path.join(output_dir, 'gsm8k_results.json'), 'w') as f:
    json.dump(results, f, indent=4)

print(f"Results saved to {os.path.join(output_dir, 'gsm8k_results.json')}")